In [1]:
from flask import Flask, request, jsonify, render_template
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from tensorflow import keras
import tempfile
import logging
import gc

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Create uploads directory if it doesn't exist
if not os.path.exists('uploads'):
    os.makedirs('uploads')

try:
    # Load model
    model_path = 'lung_cancer_model_fold_1.h5'
    logger.info(f"Loading model from: {model_path}")
    model = keras.models.load_model(model_path)
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Error loading model: {str(e)}")
    raise

def preprocess_image(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        return img_array
    except Exception as e:
        logger.error(f"Error in preprocessing image: {str(e)}")
        raise

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, 'temp_image.jpg')

    try:
        # Save and process the file
        file.save(temp_path)
        logger.info(f"Saved uploaded file temporarily as: {temp_path}")

        # Preprocess image
        logger.info("Preprocessing image")
        img_array = preprocess_image(temp_path)

        # Make prediction
        logger.info("Making prediction")
        predictions = model.predict(img_array)
        
        predicted_class = np.argmax(predictions[0])
        class_names = ['Adenocarcinoma', 'Large Cell Carcinoma', 'Normal', 'Squamous Cell Carcinoma']
        class_name = class_names[predicted_class]
        
        # Get probabilities
        probabilities = {
            'Adenocarcinoma': float(predictions[0][0]),
            'Large Cell Carcinoma': float(predictions[0][1]),
            'Normal': float(predictions[0][2]),
            'Squamous Cell Carcinoma': float(predictions[0][3])
        }

        # Clear memory
        gc.collect()

        response = {
            'class_name': class_name,
            'confidence': float(predictions[0][predicted_class]),
            'probabilities': probabilities
        }
        
        logger.info(f"Prediction successful: {response}")
        return jsonify(response)

    except Exception as e:
        logger.error(f"Error during prediction: {str(e)}")
        return jsonify({'error': f"An error occurred during prediction: {str(e)}"}), 500

    finally:
        # Clean up in finally block
        try:
            if os.path.exists(temp_path):
                os.close(os.open(temp_path, os.O_RDONLY))
                os.unlink(temp_path)
                logger.info("Temporary file cleaned up")
        except Exception as e:
            logger.error(f"Error cleaning up temporary file: {str(e)}")

# For Jupyter Notebook
from IPython.display import display, HTML
display(HTML(f'<h3>Please click here to open the application: '
            f'<a href="http://localhost:5000" target="_blank">http://localhost:5000</a></h3>'))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

INFO:__main__:Loading model from: lung_cancer_model_fold_1.h5
DEBUG:h5py._conv:Creating converter from 3 to 5
INFO:__main__:Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.34:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Nov/2024 00:47:35] "GET / HTTP/1.1" 200 -
INFO:__main__:Saved uploaded file temporarily as: C:\Users\nikhi\AppData\Local\Temp\temp_image.jpg
INFO:__main__:Preprocessing image
INFO:__main__:Making prediction


1/1 [==============================] - 2s 2s/step


INFO:__main__:Prediction successful: {'class_name': 'Normal', 'confidence': 0.34740379452705383, 'probabilities': {'Adenocarcinoma': 9.588320972397923e-05, 'Large Cell Carcinoma': 0.3110003173351288, 'Normal': 0.34740379452705383, 'Squamous Cell Carcinoma': 0.3414999842643738}}
INFO:__main__:Temporary file cleaned up
INFO:werkzeug:127.0.0.1 - - [01/Nov/2024 00:47:42] "POST /predict HTTP/1.1" 200 -
INFO:__main__:Saved uploaded file temporarily as: C:\Users\nikhi\AppData\Local\Temp\temp_image.jpg
INFO:__main__:Preprocessing image
INFO:__main__:Making prediction


1/1 [==============================] - 0s 93ms/step


INFO:__main__:Prediction successful: {'class_name': 'Normal', 'confidence': 0.3480483591556549, 'probabilities': {'Adenocarcinoma': 9.589405817678198e-05, 'Large Cell Carcinoma': 0.3111128807067871, 'Normal': 0.3480483591556549, 'Squamous Cell Carcinoma': 0.3407428562641144}}
INFO:__main__:Temporary file cleaned up
INFO:werkzeug:127.0.0.1 - - [01/Nov/2024 00:47:59] "POST /predict HTTP/1.1" 200 -
